In [1]:
# Standard imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random
import pickle
import scipy.stats
import sys
import tensorflow as tf

# Import MAVE-NN
import mavenn

# Import Logomaker for visualization
import logomaker

# Get list of demos
mavenn.run_demo()

To run a demo, execute

	>>> mavenn.run_demo(name)

where 'name' is one of the following strings:

	1. "gb1_ge_evaluation"
	2. "mpsa_ge_training"
	3. "sortseq_mpa_visualization"

Python code for each demo is located in

	/opt/homebrew/Caskroom/miniforge/base/envs/mavenn/lib/python3.8/site-packages/mavenn/examples/demos/



['gb1_ge_evaluation', 'mpsa_ge_training', 'sortseq_mpa_visualization']

In [2]:
# Synthetic CRX reporters, incorporating promoter (wild type only)

# Load data
Data = pd.read_csv("data_files/celrep_3189_mmc4.txt", sep = '\t')

# Reencode binding site arrangements so that all sequences are the same length (use O to pad)
# and each binding site is one character
max_length = 4
Padded_sequences = []
for sequence in Data['synthetic_promoter']:
    if sequence == "basal_only":
        Padded_sequence = "OOOO"
    else:
        Padded_sequence = "".join(['O']*(max_length - len(sequence)))
        for site in sequence:
            Padded_sequence = Padded_sequence + site
    Padded_sequences.append(Padded_sequence)
Data['synthetic_promoter'] = Padded_sequences
Sequence = np.concatenate([Data['synthetic_promoter'] + "P",  
                     Data['synthetic_promoter'] + "H"])
Expression = np.concatenate([np.log(Data["Rho_basal_wt"]),   
          np.log(Data["hsp68_basal_wt"])])
Data_bypromoter = pd.DataFrame({'Sequence': Sequence, 'Expression': Expression})
Data_bypromoter

,Sequence,Expression
0,OOOOP,-2.216311
1,OOOcP,-0.581550
2,OOOCP,-0.560967
3,OOOnP,-0.586571
4,OOONP,-0.568978
...,...,...
2577,xxXxH,-2.797773
2578,XXxxH,-2.415393
2579,XXxXH,-3.980350
2580,XXXxH,-3.668690


In [3]:
# Assign data to training, validation, and test sets
group = []
for i in range(len(Data_bypromoter)):
    group_num = random.random()
    if group_num < 0.1:
        group.append("test")
    elif group_num < 0.2:
        group.append("validation")
    else:
        group.append("training")
Data_bypromoter['set'] = group

# Get and report sequence length
L = len(Data_bypromoter.loc[0,'Sequence'])
print(f'Sequence length: {L:d} binding sites')

# Split dataset
trainval_df, test_df = mavenn.split_dataset(Data_bypromoter)

Data_file = open("CRX_NRL_trainval.pkl", "wb")
pickle.dump(trainval_df, Data_file)
Data_file.close()

Data_file = open("CRX_NRL_test.pkl", "wb")
pickle.dump(trainval_df, Data_file)
Data_file.close()

# Preview trainval_df
print('\ntrainval_df:')
trainval_df

Sequence length: 5 binding sites
Training set   :    2,052 observations (  79.47%)
Validation set :      253 observations (   9.80%)
Test set       :      277 observations (  10.73%)
-------------------------------------------------
Total dataset  :    2,582 observations ( 100.00%)


trainval_df:


,validation,Sequence,Expression
0,False,OOOOP,-2.216311
1,False,OOOcP,-0.581550
2,True,OOOCP,-0.560967
3,False,OOOnP,-0.586571
4,False,OOONP,-0.568978
...,...,...,...
2300,False,xxXxH,-2.797773
2301,False,XXxxH,-2.415393
2302,False,XXxXH,-3.980350
2303,False,XXXxH,-3.668690


In [4]:
# Specify alphabet
site_letters = ['N', 'n', 'C', 'c', 'R', 'r', 'X', 'x', 'P', 'H', 'O']
# Set default keyword arguments for model.Model()
default_model_kwargs = {
    'L':5,
    'alphabet':site_letters,
    'regression_type':'GE',
    'ge_noise_model_type':'SkewedT',
    'ge_heteroskedasticity_order':2
}

# Set default keyword arguments for model.fit()
default_fit_kwargs = {
    'learning_rate':.001,
    'epochs':1000,
    'batch_size':200,
    'early_stopping':True,
    'early_stopping_patience':30,
    'linear_initialization':False,
    'verbose':False
}

In [5]:
# train on all results
# Iterate over types of G-P maps

Data_file = open("CRX_NRL_trainval.pkl", "rb")
trainval_df = pickle.load(Data_file)
Data_file.close()
Test_data_file = open("CRX_NRL_test.pkl", "rb")
test_df = pickle.load(Test_data_file)
Test_data_file.close()
info_df = pd.DataFrame(columns=['name', 'gpmap', 'metric', 'I', 'dI'], index = ["additive", "neighbor", "pairwise", "blackbox"])


gpmap_types = ['additive','neighbor','pairwise','blackbox']
for i in range(25):
    random.seed(i)
    np.random.seed(i) 
    tf.random.set_seed(i)
    print(f'Training {len(gpmap_types)} models: {gpmap_types}')
    for gpmap_type in gpmap_types:

        # Set model name
        model_name = f'all_promoters_{gpmap_type}_ge_{i}'
        print('-----------------------------')
        print(f"Training '{model_name}' model...\n")

        # Copy keyword arguments
        model_kwargs = default_model_kwargs.copy()
        fit_kwargs = default_fit_kwargs.copy()

        # Modify keyword arguments based on G-P map being trained
        # Note: the need for different hyperparameters, such as batch_size
        # and learning_rate, was found by trial and error.
        if gpmap_type=='additive': pass;
        elif gpmap_type=='neighbor': pass;
        elif gpmap_type=='pairwise':
            fit_kwargs['batch_size'] = 50
        elif gpmap_type=='blackbox':
            model_kwargs['gpmap_kwargs'] = {'hidden_layer_sizes':[10]*5,
                                            'features':'pairwise'}
            fit_kwargs['learning_rate'] = 0.0005
            fit_kwargs['batch_size'] = 50
            fit_kwargs['early_stopping_patience'] = 10

        # Instantiate model using the keyword arguments in model_kwargs dict
        model = mavenn.Model(gpmap_type=gpmap_type, **model_kwargs)

        # Set training data
        model.set_data(x=trainval_df['Sequence'],
                       y=trainval_df['Expression'],
                       validation_flags=trainval_df['validation'])
    
        # Train model using the keyword arguments in fig_kwargs dict
        model.fit(**fit_kwargs)

        # Compute variational information on test data
        I_var, dI_var =  model.I_variational(x=test_df['Sequence'], y=test_df['Expression'])
        print(f'test_I_var: {I_var:.3f} +- {dI_var:.3f} bits')

        # Compute predictive information on test data
        I_pred, dI_pred = model.I_predictive(x=test_df['Sequence'], y=test_df['Expression'])
        print(f'test_I_pred: {I_pred:.3f} +- {dI_pred:.3f} bits')

        # Save model to file
        model.save(model_name)

print('Done!')

Training 4 models: ['additive', 'neighbor', 'pairwise', 'blackbox']
-----------------------------
Training 'all_promoters_additive_ge_0' model...



2023-02-17 12:25:29.617183: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


N = 2,305 observations set as training data.
Using 11.0% for validation.
Data shuffled.
Time to set data: 0.2 sec.


0epoch [00:00, ?epoch/s]

Training time: 43.6 seconds
test_I_var: 0.588 +- 0.033 bits
test_I_pred: 0.790 +- 0.025 bits
Model saved to these files:
	all_promoters_additive_ge_0.pickle
	all_promoters_additive_ge_0.h5
-----------------------------
Training 'all_promoters_neighbor_ge_0' model...

N = 2,305 observations set as training data.
Using 11.0% for validation.
Data shuffled.
Time to set data: 0.175 sec.


...

Training time: 29.4 seconds
test_I_var: 0.951 +- 0.043 bits
test_I_pred: 1.114 +- 0.021 bits
Model saved to these files:
	all_promoters_neighbor_ge_24.pickle
	all_promoters_neighbor_ge_24.h5
-----------------------------
Training 'all_promoters_pairwise_ge_24' model...

N = 2,305 observations set as training data.
Using 11.0% for validation.
Data shuffled.
Time to set data: 0.117 sec.


0epoch [00:00, ?epoch/s]

Training time: 20.9 seconds
test_I_var: 1.691 +- 0.038 bits
test_I_pred: 1.803 +- 0.034 bits
Model saved to these files:
	all_promoters_pairwise_ge_24.pickle
	all_promoters_pairwise_ge_24.h5
-----------------------------
Training 'all_promoters_blackbox_ge_24' model...

N = 2,305 observations set as training data.
Using 11.0% for validation.
Data shuffled.
Time to set data: 0.12 sec.


0epoch [00:00, ?epoch/s]

Training time: 13.4 seconds
test_I_var: 1.703 +- 0.033 bits
test_I_pred: 2.636 +- 0.028 bits
Model saved to these files:
	all_promoters_blackbox_ge_24.pickle
	all_promoters_blackbox_ge_24.h5
Done!


In [6]:
# Jamie's synthetic CRX reporters in CRX null retina

# Load data
Data = pd.read_csv("celrep_3189_mmc4.txt", sep = '\t')

# Reencode binding site arrangements so that all sequences are the same length (use O to pad)
# and each binding site is one character
max_length = 4
Padded_sequences = []
for sequence in Data['synthetic_promoter']:
    if sequence == "basal_only":
        Padded_sequence = "OOOO"
    else:
        Padded_sequence = "".join(['O']*(max_length - len(sequence))) + sequence 
    Padded_sequences.append(Padded_sequence)
Data['synthetic_promoter'] = Padded_sequences
Data['Rho_basal_crxnull'] = np.log(Data['Rho_basal_crxnull'])
Data

,synthetic_promoter,Rho_basal_wt,Rho_basal_crxnull,hsp68_basal_wt,NRL_motif,CRX_affinity_score
0,OOOO,0.109011,-3.566171,0.021857,0,0.000000
1,OOOc,0.559031,-0.843686,0.023614,0,0.999877
2,OOOC,0.570657,-0.908113,0.094922,0,0.999877
3,OOOn,0.556231,-1.350185,0.011144,1,0.000000
4,OOON,0.566104,-1.652386,0.031881,1,0.000000
...,...,...,...,...,...,...
1286,xxXx,0.248542,-1.496065,0.060946,0,0.299828
1287,XXxx,0.671732,-0.221630,0.089332,0,0.299828
1288,XXxX,0.539617,-0.694057,0.018679,0,0.299828
1289,XXXx,0.484277,-0.661881,0.025510,0,0.299828


In [7]:
# Assign data to training, validation, and test sets
group = []
for i in range(len(Data)):
    group_num = random.random()
    if group_num < 0.1:
        group.append("test")
    elif group_num < 0.2:
        group.append("validation")
    else:
        group.append("training")
Data['set'] = group

# Get and report sequence length
L = len(Data.loc[0,'synthetic_promoter'])
print(f'Sequence length: {L:d} binding sites')

# Split dataset
trainval_df, test_df = mavenn.split_dataset(Data)

Data_file = open("CRX_null_trainval.pkl", "wb")
pickle.dump(trainval_df, Data_file)
Data_file.close()

Data_file = open("CRX_null_test.pkl", "wb")
pickle.dump(trainval_df, Data_file)
Data_file.close()

# Preview trainval_df
print('\ntrainval_df:')
trainval_df

Sequence length: 4 binding sites
Training set   :    1,031 observations (  79.86%)
Validation set :      139 observations (  10.77%)
Test set       :      121 observations (   9.37%)
-------------------------------------------------
Total dataset  :    1,291 observations ( 100.00%)


trainval_df:


,validation,synthetic_promoter,Rho_basal_wt,Rho_basal_crxnull,hsp68_basal_wt,NRL_motif,CRX_affinity_score
0,False,OOOO,0.109011,-3.566171,0.021857,0,0.000000
1,False,OOOc,0.559031,-0.843686,0.023614,0,0.999877
2,True,OOOC,0.570657,-0.908113,0.094922,0,0.999877
3,False,OOOn,0.556231,-1.350185,0.011144,1,0.000000
4,True,OOON,0.566104,-1.652386,0.031881,1,0.000000
...,...,...,...,...,...,...,...
1165,False,xxXx,0.248542,-1.496065,0.060946,0,0.299828
1166,False,XXxx,0.671732,-0.221630,0.089332,0,0.299828
1167,False,XXxX,0.539617,-0.694057,0.018679,0,0.299828
1168,False,XXXx,0.484277,-0.661881,0.025510,0,0.299828


In [8]:
Data_file = open("CRX_null_trainval.pkl", "rb")
trainval_df = pickle.load(Data_file)
Data_file.close()

Data_file = open("CRX_null_test.pkl", "rb")
test_df = pickle.load(Data_file)
Data_file.close()

# train on CRX null results
site_letters = ['N', 'n', 'C', 'c', 'R', 'r', 'X', 'x', 'O']
# Set default keyword arguments for model.Model()
default_model_kwargs = {
    'L':4,
    'alphabet':site_letters,
    'regression_type':'GE',
    'ge_noise_model_type':'SkewedT',
    'ge_heteroskedasticity_order':2
}

# Set default keyword arguments for model.fit()
default_fit_kwargs = {
    'learning_rate':.001,
    'epochs':1000,
    'batch_size':200,
    'early_stopping':True,
    'early_stopping_patience':30,
    'linear_initialization':False,
    'verbose':False
}

# Iterate over types of G-P maps
gpmap_types = ['additive','neighbor','pairwise','blackbox']
print(f'Training {len(gpmap_types)} models: {gpmap_types}')
for i in range(20):
    random.seed(i)
    np.random.seed(i) 
    tf.random.set_seed(i)
    for gpmap_type in gpmap_types:

        # Set model name
        model_name = f'crxnull_{gpmap_type}_ge_{i}'
        print('-----------------------------')
        print(f"Training '{model_name}' model...\n")

        # Copy keyword arguments
        model_kwargs = default_model_kwargs.copy()
        fit_kwargs = default_fit_kwargs.copy()

        # Modify keyword arguments based on G-P map being trained
        # Note: the need for different hyperparameters, such as batch_size
        # and learning_rate, was found by trial and error.
        if gpmap_type=='additive': pass;
        elif gpmap_type=='neighbor': pass;
        elif gpmap_type=='pairwise':
            fit_kwargs['batch_size'] = 50
        elif gpmap_type=='blackbox':
            model_kwargs['gpmap_kwargs'] = {'hidden_layer_sizes':[10]*5,
                                            'features':'pairwise'}
            fit_kwargs['learning_rate'] = 0.0005
            fit_kwargs['batch_size'] = 50
            fit_kwargs['early_stopping_patience'] = 10

        # Instantiate model using the keyword arguments in model_kwargs dict
        model = mavenn.Model(gpmap_type=gpmap_type, **model_kwargs)

        # Set training data
        model.set_data(x=trainval_df['synthetic_promoter'],
                       y=trainval_df['Rho_basal_crxnull'],
                       validation_flags=trainval_df['validation'])

        # Train model using the keyword arguments in fig_kwargs dict
        model.fit(**fit_kwargs)

        # Compute variational information on test data
        I_var, dI_var =  model.I_variational(x=test_df['synthetic_promoter'], y=test_df['Rho_basal_crxnull'])
        print(f'test_I_var: {I_var:.3f} +- {dI_var:.3f} bits')

        # Compute predictive information on test data
        I_pred, dI_pred = model.I_predictive(x=test_df['synthetic_promoter'], y=test_df['Rho_basal_crxnull'])
        print(f'test_I_pred: {I_pred:.3f} +- {dI_pred:.3f} bits')

        # Save model to file
        model.save(model_name)

print('Done!')

Training 4 models: ['additive', 'neighbor', 'pairwise', 'blackbox']
-----------------------------
Training 'crxnull_additive_ge_0' model...

N = 1,170 observations set as training data.
Using 11.9% for validation.
Data shuffled.
Time to set data: 0.0699 sec.


0epoch [00:00, ?epoch/s]

Training time: 24.3 seconds
test_I_var: 0.724 +- 0.043 bits
test_I_pred: 0.779 +- 0.024 bits
Model saved to these files:
	crxnull_additive_ge_0.pickle
	crxnull_additive_ge_0.h5
-----------------------------
Training 'crxnull_neighbor_ge_0' model...

N = 1,170 observations set as training data.
Using 11.9% for validation.
Data shuffled.
Time to set data: 0.0623 sec.


0epoch [00:00, ?epoch/s]

Training time: 15.6 seconds
test_I_var: 1.113 +- 0.050 bits
test_I_pred: 1.248 +- 0.039 bits
Model saved to these files:
	crxnull_neighbor_ge_0.pickle
	crxnull_neighbor_ge_0.h5
-----------------------------
Training 'crxnull_pairwise_ge_0' model...

N = 1,170 observations set as training data.
Using 11.9% for validation.
Data shuffled.
Time to set data: 0.0663 sec.


...

Training time: 17.4 seconds
test_I_var: 1.120 +- 0.046 bits
test_I_pred: 1.214 +- 0.041 bits
Model saved to these files:
	crxnull_neighbor_ge_19.pickle
	crxnull_neighbor_ge_19.h5
-----------------------------
Training 'crxnull_pairwise_ge_19' model...

N = 1,170 observations set as training data.
Using 11.9% for validation.
Data shuffled.
Time to set data: 0.0394 sec.


0epoch [00:00, ?epoch/s]

Training time: 12.8 seconds
test_I_var: 1.904 +- 0.056 bits
test_I_pred: 1.976 +- 0.030 bits
Model saved to these files:
	crxnull_pairwise_ge_19.pickle
	crxnull_pairwise_ge_19.h5
-----------------------------
Training 'crxnull_blackbox_ge_19' model...

N = 1,170 observations set as training data.
Using 11.9% for validation.
Data shuffled.
Time to set data: 0.0402 sec.


0epoch [00:00, ?epoch/s]

Training time: 8.4 seconds
test_I_var: 1.448 +- 0.042 bits
test_I_pred: 2.776 +- 0.041 bits
Model saved to these files:
	crxnull_blackbox_ge_19.pickle
	crxnull_blackbox_ge_19.h5
Done!


In [9]:
# Ryan's building block library

Data = pd.read_csv("data_files/cdnrm_mpra_library.txt", sep = '\t', header = None, names = ['Sequence', 'Expression'])

# Reencode binding site arrangements so that all sequences are the same length (use O to pad)
# and each binding site is one character
max_length = 5
Padded_sequences = []
for sequence in Data['Sequence']:
    building_blocks = sequence.split("_")[1]
    encoding = ""
    letter = ""
    for i in np.linspace(0, 8, 5):
        i = int(i)
        if building_blocks[i:(i + 2)] == "Ch":
            encoding = encoding + "C"
        elif building_blocks[i:(i + 2)] == "Cm":
            encoding = encoding + "R"
        elif building_blocks[i:(i + 2)] == "Cl":
            encoding = encoding + "X"
        elif building_blocks[i:(i + 2)] == "Dh":
            encoding = encoding + "D"
        elif building_blocks[i:(i + 2)] == "Dm":
            encoding = encoding + "E"
        elif building_blocks[i:(i + 2)] == "Nh":
            encoding = encoding + "N"
        elif building_blocks[i:(i + 2)] == "Nm":
            encoding = encoding + "L"
        elif building_blocks[i:(i + 2)] == "Rh":
            encoding = encoding + "O"
        elif building_blocks[i:(i + 2)] == "Rm":
            encoding = encoding + "B"
        elif building_blocks[i:(i + 2)] == "Mh":
            encoding = encoding + "A"
        elif building_blocks[i:(i + 2)] == "Mm":
            encoding = encoding + "Z"
    Padded_sequences.append(encoding)
Data['Sequence'] = Padded_sequences
Data
#Sequence = np.concatenate([Data['synthetic_promoter'] + "P", 
#                     Data['synthetic_promoter'] + "p", 
#                     Data['synthetic_promoter'] + "H"])
#Expression = np.concatenate([Data["Rho_basal_wt"],  
#          Data["Rho_basal_crxnull"], 
#          Data["hsp68_basal_wt"]])
#Data_bypromoter = pd.DataFrame({'Sequence': Sequence, 'Expression': Expression})
#Data_bypromoter

,Sequence,Expression
0,CCCDZ,3.281311
1,CCCEA,-0.350084
2,CCCEN,-0.169063
3,CCCAE,0.420798
4,CCCAN,1.890108
...,...,...
6593,BOACC,4.623494
6594,BOACR,4.390299
6595,BOARC,4.523007
6596,BONCR,4.696074


In [10]:
# Assign data to training, validation, and test sets
group = []
for i in range(len(Data)):
    group_num = random.random()
    if group_num < 0.1:
        group.append("test")
    elif group_num < 0.2:
        group.append("validation")
    else:
        group.append("training")
Data['set'] = group

# Get and report sequence length
L = len(Data.loc[0,'Sequence'])
print(f'Sequence length: {L:d} binding sites')

# Split dataset
trainval_df, test_df = mavenn.split_dataset(Data)

Data_file = open("Ryan_trainval.pkl", "wb")
pickle.dump(trainval_df, Data_file)
Data_file.close()

Data_file = open("Ryan_test.pkl", "wb")
pickle.dump(test_df, Data_file)
Data_file.close()

# Preview trainval_df
print('\ntrainval_df:')
trainval_df

Sequence length: 5 binding sites
Training set   :    5,274 observations (  79.93%)
Validation set :      658 observations (   9.97%)
Test set       :      666 observations (  10.09%)
-------------------------------------------------
Total dataset  :    6,598 observations ( 100.00%)


trainval_df:


,validation,Sequence,Expression
0,False,CCCDZ,3.281311
1,False,CCCEA,-0.350084
2,False,CCCEN,-0.169063
3,False,CCCAE,0.420798
4,False,CCCAN,1.890108
...,...,...,...
5927,False,BOACC,4.623494
5928,False,BOACR,4.390299
5929,False,BOARC,4.523007
5930,False,BONCR,4.696074


In [ ]:
Data_file = open("Ryan_trainval.pkl", "rb")
trainval_df = pickle.load(Data_file)
Data_file.close()

Data_file = open("Ryan_test.pkl", "rb")
test_df = pickle.load(Data_file)
Data_file.close()

# train on all results
site_letters = ['C', 'R', 'X', 'D', 'E', 'N', 'L', 'O', 'B', 'A', 'Z']
# Set default keyword arguments for model.Model()
default_model_kwargs = {
    'L':5,
    'alphabet':site_letters,
    'regression_type':'GE',
    'ge_noise_model_type':'SkewedT',
    'ge_heteroskedasticity_order':2
}

# Set default keyword arguments for model.fit()
default_fit_kwargs = {
    'learning_rate':.001,
    'epochs':500,
    'batch_size':200,
    'early_stopping':True,
    'early_stopping_patience':30,
    'linear_initialization':False,
    'verbose':False
}

# Iterate over types of G-P maps
gpmap_types = ['additive','neighbor','pairwise','blackbox']
print(f'Training {len(gpmap_types)} models: {gpmap_types}')

for i in range(50):
    random.seed(i)
    np.random.seed(i) 
    tf.random.set_seed(i)
    for gpmap_type in gpmap_types:

        # Set model name
        model_name = f'CDNRM_{gpmap_type}_ge_{i}'
        print('-----------------------------')
        print(f"Training '{model_name}' model...\n")

        # Copy keyword arguments
        model_kwargs = default_model_kwargs.copy()
        fit_kwargs = default_fit_kwargs.copy()

        # Modify keyword arguments based on G-P map being trained
        # Note: the need for different hyperparameters, such as batch_size
        # and learning_rate, was found by trial and error.
        if gpmap_type=='additive': pass;
        elif gpmap_type=='neighbor': pass;
        elif gpmap_type=='pairwise':
            fit_kwargs['batch_size'] = 50
        elif gpmap_type=='blackbox':
            model_kwargs['gpmap_kwargs'] = {'hidden_layer_sizes':[10]*5,
                                        'features':'pairwise'}
            fit_kwargs['learning_rate'] = 0.0005
            fit_kwargs['batch_size'] = 50
            fit_kwargs['early_stopping_patience'] = 10

        # Instantiate model using the keyword arguments in model_kwargs dict
        model = mavenn.Model(gpmap_type=gpmap_type, **model_kwargs)

        # Set training data
        model.set_data(x=trainval_df['Sequence'],
                       y=trainval_df['Expression'],
                       validation_flags=trainval_df['validation'])
    
        # Train model using the keyword arguments in fig_kwargs dict
        model.fit(**fit_kwargs)

        # Compute variational information on test data
        I_var, dI_var =  model.I_variational(x=test_df['Sequence'], y=test_df['Expression'])
        print(f'test_I_var: {I_var:.3f} +- {dI_var:.3f} bits')

        # Compute predictive information on test data
        I_pred, dI_pred = model.I_predictive(x=test_df['Sequence'], y=test_df['Expression'])
        print(f'test_I_pred: {I_pred:.3f} +- {dI_pred:.3f} bits')

        # Save model to file
        model.save(model_name)

print('Done!')

Training 4 models: ['additive', 'neighbor', 'pairwise', 'blackbox']
-----------------------------
Training 'CDNRM_additive_ge_0' model...

N = 5,932 observations set as training data.
Using 11.1% for validation.
Data shuffled.
Time to set data: 0.168 sec.


0epoch [00:00, ?epoch/s]

Training time: 23.3 seconds
test_I_var: 1.406 +- 0.072 bits
test_I_pred: 1.527 +- 0.040 bits
Model saved to these files:
	CDNRM_additive_ge_0.pickle
	CDNRM_additive_ge_0.h5
-----------------------------
Training 'CDNRM_neighbor_ge_0' model...

N = 5,932 observations set as training data.
Using 11.1% for validation.
Data shuffled.
Time to set data: 0.154 sec.


0epoch [00:00, ?epoch/s]

Training time: 56.9 seconds
test_I_var: 2.073 +- 0.062 bits
test_I_pred: 2.198 +- 0.036 bits
Model saved to these files:
	CDNRM_neighbor_ge_0.pickle
	CDNRM_neighbor_ge_0.h5
-----------------------------
Training 'CDNRM_pairwise_ge_0' model...

N = 5,932 observations set as training data.
Using 11.1% for validation.
Data shuffled.
Time to set data: 0.148 sec.


...

0epoch [00:00, ?epoch/s]

Training time: 53.7 seconds
test_I_var: 2.067 +- 0.062 bits
test_I_pred: 2.173 +- 0.039 bits
Model saved to these files:
	CDNRM_neighbor_ge_23.pickle
	CDNRM_neighbor_ge_23.h5
-----------------------------
Training 'CDNRM_pairwise_ge_23' model...

N = 5,932 observations set as training data.
Using 11.1% for validation.
Data shuffled.
Time to set data: 0.253 sec.


0epoch [00:00, ?epoch/s]

Training time: 67.9 seconds
test_I_var: 2.522 +- 0.064 bits
test_I_pred: 2.634 +- 0.044 bits
Model saved to these files:
	CDNRM_pairwise_ge_23.pickle
	CDNRM_pairwise_ge_23.h5
-----------------------------
Training 'CDNRM_blackbox_ge_23' model...

N = 5,932 observations set as training data.
Using 11.1% for validation.
Data shuffled.
Time to set data: 0.32 sec.


0epoch [00:00, ?epoch/s]